## XML exercise


1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [7]:
import pandas as pd
import xmltodict
# open a file and convert to a dictionary
with open("./data/mondial.xml") as xml_file:
    data_dict = xmltodict.parse(xml_file.read())

In [8]:
data_xml = data_dict['mondial']

In [10]:
df = pd.json_normalize(data_xml)

In [11]:
df.head()

,country,continent,organization,sea,river,lake,island,mountain,desert,airport
0,"[{'@car_code': 'AL', '@area': '28750', '@capit...","[{'@id': 'europe', 'name': 'Europe', 'area': '...","[{'@id': 'org-AfDB', '@headq': 'cty-CI-2', 'na...","[{'@id': 'sea-Atlantic', '@country': 'F E GBZ ...","[{'@id': 'river-Thjorsa', '@country': 'IS', 'n...","[{'@id': 'lake-Inarisee', '@country': 'SF', 'n...","[{'@id': 'island-Svalbard', '@country': 'SVAX'...","[{'@id': 'mount-GunnbjornFjeld', '@country': '...","[{'@id': 'desert-Kalahari', '@country': 'NAM R...","[{'@iatacode': 'HEA', '@city': 'cty-Afghanista..."


## 1

In [15]:
pd.json_normalize(data_xml, record_path = ['country'])[['name','infant_mortality']].sort_values('infant_mortality').head(10)

,name,infant_mortality
38,Monaco,1.81
30,Romania,10.16
153,Fiji,10.2
69,Brunei,10.48
132,Grenada,10.5
237,Mauritius,10.59
124,Panama,10.7
243,Seychelles,10.77
102,United Arab Emirates,10.92
113,Barbados,10.93


## 2

In [233]:
# firstly, we need to extract provices
provinces = pd.json_normalize(data_xml['country'])[['name','province']]
provinces = provinces[provinces.province.notnull()]

In [234]:
# from provinces, we extract cities
cities = pd.json_normalize(provinces[['province']].to_dict(orient='records'), record_path = 'province')
cities = cities[cities.city.notnull()][['population','city']]

In [235]:
cities.head()

,population,city
0,"[{'@measured': 'census', '@year': '1991', '#te...","[{'@id': 'cty-Greece-Komotini', '@country': 'G..."
1,"[{'@measured': 'census', '@year': '1991', '#te...","[{'@id': 'cty-Greece-Athens', '@country': 'GR'..."
7,"[{'@measured': 'census', '@year': '1991', '#te...","[{'@id': 'cty-Greece-7', '@country': 'GR', '@p..."
8,"[{'@measured': 'census', '@year': '1991', '#te...","[{'@id': 'cty-Greece-Ermoupoli', '@country': '..."
10,"[{'@measured': 'census', '@year': '1991', '#te...","[{'@id': 'cty-Greece-Lamia', '@country': 'GR',..."


In [236]:
# now we need to extract populations for each city
populations = pd.json_normalize(cities.to_dict(orient='records'), record_path = ['city'])[['name','population']]
populations.tail()

,name,population
1986,M'Bour,"[{'@year': '2002', '@measured': 'census', '#te..."
1987,Kampala,"[{'@year': '1991', '@measured': 'census', '#te..."
1988,Mukono,"[{'@year': '1969', '@measured': 'census', '#te..."
1989,"[Nacala Porto, Nacala]","[{'@year': '1991', '#text': '125208'}, {'@year..."
1990,Matola,"[{'@year': '1991', '#text': '337239'}, {'@year..."


In [230]:
# we convert cities and their populations back to dict (json) so we can call json_normalize on it again
all_populations = pd.json_normalize(populations.to_dict(orient='records'), record_path = 'population', meta = 'name')
all_populations["@year"] = all_populations["@year"].astype(int)
all_populations["#text"] = all_populations["#text"].astype(int)
all_populations["name"] = all_populations["name"].astype(str)

In [231]:
all_populations.head()

,@year,#text,@measured,name
0,1981,56705,NaN,Kavala
1,1991,60187,census,Kavala
2,2001,63774,census,Kavala
3,2011,58790,census,Kavala
4,1981,885737,NaN,"['Athina', 'Athens']"


In [232]:
# dataframe populations is sorted by city name and year, 
# so we can simply get rid of duplicate cities and take the last value
all_populations.drop_duplicates(subset=['name'], keep='last').sort_values("#text", ascending=False).head(10)

,@year,#text,@measured,name
2601,2017,14916456,census,"['Karachi', 'Karāchi']"
5835,2016,13745000,projection,Lagos
2714,2011,12442373,census,Mumbai
5410,2010,11152344,census,São Paulo
2517,2017,11126285,census,Lahore
2176,2010,11071424,census,Guangzhou
2183,2010,10358381,census,Shenzhen
2219,2010,9785388,census,Wuhan
2470,2016,8693706,census,Tehran
3062,2012,8591695,estimate,Tokyo


## 3

In [78]:
rivers = pd.json_normalize(data_xml, record_path = ['river'])[['name','length','@country']]

In [80]:
rivers.length = rivers.length.astype(float)

In [82]:
rivers.sort_values('length', ascending=False).head(1)

,name,length,@country
214,Yangtze,6380.0,CN


In [85]:
lakes = pd.json_normalize(data_xml, record_path = ['lake'])[['name','area','@country']]

In [86]:
lakes.area = lakes.area.astype(float)

In [87]:
lakes.sort_values('area', ascending=False).head(1)

,name,area,@country
59,Caspian Sea,386400.0,R AZ KAZ IR TM


In [91]:
airports = pd.json_normalize(data_xml, record_path = ['airport'])[['name','elevation','@country']]

In [92]:
airports.elevation = airports.elevation.astype(float)

In [93]:
airports.sort_values('elevation', ascending=False).head(1)

,name,elevation,@country
81,El Alto Intl,4063.0,BOL
